In [1]:
import sys,os
sys.path.append(os.path.dirname(os.getcwd()))
from my_libs_py3 import *
pd.set_option("display.max_rows",100)
import bson
# import cudf

/home/ken/anaconda3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
# df = cudf.DataFrame({"a":{1, 2, 3, 4, 5}})

# data = data.drop("_id",axis=1)

# data["Refresh_Date"] = data["Refresh_Date"].apply(lambda x: x.strftime("%Y-%m-%d"))


In [3]:
mongod = mongo("all_symbol","screener")
## This line gets the max refresh_date
tos = pd.DataFrame(mongod.conn.table.find({},{"Refresh_Date":1}).sort("Refresh_Date",-1).limit(1))["Refresh_Date"].iloc[0]
all_ticker = pd.DataFrame(mongod.conn.table.find({"Refresh_Date":tos}))
all_ticker = all_ticker[all_ticker.Volume > all_ticker.Volume.quantile(0.5)]
all_ticker = all_ticker[all_ticker["Institutional Ownership"] > all_ticker["Institutional Ownership"].quantile(0.5)]

all_ticker = all_ticker.Ticker.to_list()


Connection Successful


In [7]:
mongod = mongo("all_symbol","screenerModel")

## This line gets the max refresh_date
tos = pd.DataFrame(mongod.conn.table.find({},{"Refresh_Date":1,"_id":0}).sort("Refresh_Date",-1).limit(1))["Refresh_Date"].iloc[0]

fun_table = pd.DataFrame(mongod.conn.table.find({"Refresh_Date":tos},{"_id":0}))

target = ["P/Cash","Analyst Recom","P/S","Total Debt/Equity","P/Free Cash Flow","P/E","Insider Ownership",
         "Gross Margin","Current Ratio","Sales growth quarter over quarter","Profit Margin","Quick Ratio","Performance (Week)","Institutional Ownership","EPS (ttm)","Operating Margin"]

fun_table["sum_rank"] = pd.DataFrame.sum(fun_table[target],axis=1,skipna=True,numeric_only=True)

fun_table["avg_rank"] = pd.DataFrame.mean(fun_table[target],axis=1,skipna=True,numeric_only=True)

# fun_table = fun_table.sort_values("avg_rank")

for i in fun_table.index:
    relative_rank = fun_table.loc[i, "avg_rank"] / fun_table.loc[
        fun_table.Sector == fun_table.loc[i, "Sector"], "Sector"].count()
    #     relative_rank = fun_table.loc[i,"avg_rank"]/fun_table.loc[fun_table.Sector == fun_table.loc[i,"Sector"],"Market Cap"].mean()
    #     relative_rank = fun_table.loc[i,"avg_rank"]*(1- (1/fun_table.loc[fun_table.Sector == fun_table.loc[i,"Sector"],"Market Cap"].mean()))
    #     relative_rank = fun_table.loc[i,"avg_rank"]/fun_table.loc[fun_table.Sector == fun_table.loc[i,"Sector"],"Market Cap"].mean()

    fun_table.loc[i, "relative_rank"] = relative_rank
#     fun_table.loc[i,"relative_rank"] = fun_table.loc[i,"avg_rank"]

fun_table = fun_table.sort_values("relative_rank")


target_list = fun_table.Ticker[:20]

Connection Successful


In [12]:
mongod_1 = mongo("all_symbol","screener_tos")

Connection Successful


In [15]:
pd.DataFrame(mongod_1.conn.table.find({"Ticker":"QURE"})).T

,0
_id,605051b8e5bfa2513a14e8ad
20-Day Simple Moving Average,-0.0399
200-Day Simple Moving Average,-0.2097
50-Day Simple Moving Average,-0.0807
52-Week High,-0.5267
52-Week Low,0.0995
Analyst Recom,1.7000
Average Volume,553.4700
Beta,1.3300
Change,-0.0041


In [9]:
fun_table[:20]

,Ticker,Sector,Industry,Refresh_Date,P/S,P/Cash,P/E,P/Free Cash Flow,Analyst Recom,Total Debt/Equity,Performance (Week),Current Ratio,Quick Ratio,Sales growth quarter over quarter,Profit Margin,EPS (ttm),Operating Margin,Insider Ownership,Institutional Ownership,Gross Margin,sum_rank,avg_rank,relative_rank
931,ECCC,Financial,Asset Management,2022-01-27 13:01:01.730000+00:00,NaN,NaN,134.0000,NaN,NaN,NaN,325.0000,NaN,NaN,64.0000,NaN,161.0000,NaN,NaN,NaN,NaN,684.0000,171.0000,0.1644
988,ENVA,Financial,Credit Services,2022-01-27 13:01:01.730000+00:00,114.0000,459.0000,106.0000,50.0000,155.0000,573.0000,146.0000,8.0000,8.0000,43.0000,176.0000,54.0000,342.0000,333.0000,69.0000,NaN,"2,636.0000",175.7333,0.1690
2626,SBNYP,Financial,Banks - Regional,2022-01-27 13:01:01.730000+00:00,NaN,NaN,22.0000,NaN,NaN,NaN,578.0000,NaN,NaN,94.0000,NaN,43.0000,NaN,NaN,NaN,NaN,737.0000,184.2500,0.1772
2153,NTIP,Technology,Communication Equipment,2022-01-27 13:01:01.730000+00:00,123.0000,4.0000,13.0000,1.0000,NaN,76.0000,29.0000,3.0000,3.0000,10.0000,16.0000,260.0000,5.0000,111.0000,291.0000,75.0000,"1,020.0000",68.0000,0.1889
1852,MBINN,Financial,Banks - Regional,2022-01-27 13:01:01.730000+00:00,NaN,NaN,112.0000,NaN,NaN,NaN,193.0000,NaN,NaN,363.0000,NaN,124.0000,NaN,NaN,NaN,NaN,792.0000,198.0000,0.1904
1016,ESGR,Financial,Insurance - Diversified,2022-01-27 13:01:01.730000+00:00,249.0000,360.0000,191.0000,5.0000,NaN,461.0000,24.0000,NaN,NaN,NaN,15.0000,7.0000,NaN,511.0000,169.0000,NaN,"1,992.0000",199.2000,0.1915
3208,WFG,Basic Materials,Lumber & Wood Production,2022-01-27 13:01:01.730000+00:00,60.0000,25.0000,9.0000,5.0000,4.0000,35.0000,86.0000,52.0000,47.0000,70.0000,14.0000,1.0000,15.0000,33.0000,67.0000,11.0000,534.0000,33.3750,0.1940
2489,QURE,Healthcare,Biotechnology,2022-01-27 13:01:01.730000+00:00,64.0000,10.0000,8.0000,3.0000,57.0000,92.0000,197.0000,15.0000,14.0000,139.0000,3.0000,42.0000,9.0000,114.0000,11.0000,8.0000,786.0000,49.1250,0.1989
2752,SOHU,Communication Services,Internet Content & Information,2022-01-27 13:01:01.730000+00:00,46.0000,2.0000,22.0000,NaN,NaN,22.0000,8.0000,23.0000,22.0000,22.0000,NaN,34.0000,NaN,NaN,82.0000,NaN,283.0000,28.3000,0.2036
2478,QFIN,Financial,Credit Services,2022-01-27 13:01:01.730000+00:00,108.0000,423.0000,110.0000,NaN,30.0000,211.0000,842.0000,53.0000,51.0000,159.0000,229.0000,190.0000,406.0000,69.0000,347.0000,35.0000,"3,263.0000",217.5333,0.2092


In [6]:
    
my_future = future()



symbol =  "VX" + "/" + my_future.monthCode(datetime.now().month+1) + str(int(datetime.now().year)+1)[-1:] if datetime.now().month + 1 > 12 \
          else "VX" + "/" + my_future.monthCode(datetime.now().month + 1) + str(datetime.now().year)[-1:]

new_frame = pd.DataFrame({"TimeStamp":datetime(datetime.now().year,datetime.now().month,datetime.now().day),"Close":[my_future.get_future_price(datetime.now().year,datetime.now().month)],"VIX_Day_Roll":((my_future.get_future_price(datetime.now().year,datetime.now().month) - realtimequote("^VIX").price.values[0])/my_future.get_historic_data(my_future.get_future_expiration(datetime.now().year,datetime.now().month)).Close.std())/30, "symbol":symbol}).reset_index()
new_frame = new_frame.drop("index",axis = 1)


# table_name = "VX" + my_future.monthCode(datetime.now().month+1)+str(datetime.now().year)[-2:]

# mongod.conn.to_sql(new_frame,"future",symbol,if_exists="append")
mongo_conn = connect_mongo("future",symbol)
mongo_conn.update_to_mongo(new_frame,"TimeStamp")


#     new_frame=json.loads(new_frame.T.to_json()).values()[0]

#     mongod.db["VX" + my_future.monthCode(datetime.now().month+1)+str(datetime.now().year)[:-2]].update_many({"TimeStamp":new_frame["TimeStamp"]},{"$set":new_frame},upsert=True)






VX/F2
Message: 'geckodriver' executable needs to be in PATH. 

VX/F2
Message: 'geckodriver' executable needs to be in PATH. 

VX/F2
Message: 'geckodriver' executable needs to be in PATH. 

VX/F2
Message: 'geckodriver' executable needs to be in PATH. 

VX/F2
Message: 'geckodriver' executable needs to be in PATH. 

VX/F2
Message: 'geckodriver' executable needs to be in PATH. 



TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [62]:
def pair_trade_smaple():
    mongod = mongo("all_symbol","pair_trade_sharp_2021")
    candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}))
    top_return = candid.describe().loc["75%","Avg_Return"]
    top_sharp = candid.describe().loc["75%","Sharp_Ratio"]

    pipeline = [
        {"$match":{"End_Value":{"$gt":2000},"Sharp_Ratio":{"$gt":top_sharp},"Avg_Return":{"$gt":top_return}}},
        {"$sample":{"size":10}}
    ]

    candid = pd.DataFrame(mongod.conn.table.aggregate(pipeline))

    return candid

In [64]:
pair_trade_smaple()

Connection Successful


,_id,Ticker_1,Ticker_2,End_Value,Avg_Return,Sharp_Ratio,Min_Return,Max_Return,Refresh_Date
0,612c5e83ffb50dc4b25489be,ALRM,B,"2,240.4159",0.0005,0.0649,-0.0389,0.0413,2021-08-22 00:00:00+00:00
1,6122eed5ffb50dc4b24dd0c6,AAT,GNW,"2,544.0665",0.0010,0.0680,-0.0586,0.1562,2021-08-22 00:00:00+00:00
2,6126496dffb50dc4b2502fe2,ADSK,TIG,"2,577.8298",0.0010,0.1031,-0.0495,0.0379,2021-08-22 00:00:00+00:00
3,612d09b4ffb50dc4b255023a,AMBC,RTPYU,"2,167.0800",0.0007,0.1373,-0.0204,0.0282,2021-08-22 00:00:00+00:00
4,61268243ffb50dc4b25057bc,ADTN,HMHC,"2,442.3220",0.0008,0.0748,-0.0694,0.0511,2021-08-22 00:00:00+00:00
5,61244c3effb50dc4b24ec414,ACIW,SI,"2,755.4998",0.0013,0.1275,-0.0140,0.1151,2021-08-22 00:00:00+00:00
6,612406f6ffb50dc4b24e9344,ACC,UNIT,"2,337.6120",0.0006,0.0857,-0.0287,0.0327,2021-08-22 00:00:00+00:00
7,612b22c9ffb50dc4b253a596,ALGN,CORT,"2,448.4300",0.0008,0.0708,-0.0634,0.0589,2021-08-22 00:00:00+00:00
8,612b2521ffb50dc4b253a738,ALGN,DOC,"2,431.3901",0.0008,0.0777,-0.0634,0.0589,2021-08-22 00:00:00+00:00
9,6123215bffb50dc4b24df3ca,ABCB,RDWR,"2,133.5067",0.0003,0.1111,-0.0129,0.0301,2021-08-22 00:00:00+00:00


In [42]:
trade = self_pair_trade("ADP","APR").iloc[-1]

All price data of Close is actually Adj Close
Connection Successful
ADP
('Finished', 'ADP')
All price data of Close is actually Adj Close
Connection Successful
APR
('Finished', 'APR')
All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [8]:
trade.iloc[-1]["total_value"]

2571.852227

In [24]:
np.log(trade["total_value"].shift(-1)/trade["total_value"]).mean()

0.0010140289098981633

In [23]:
trade["total_value"].head(7)

TimeStamp
2020-08-26   2,000.0000
2020-08-27   2,000.0000
2020-08-28   2,000.0000
2020-08-31   2,000.0000
2020-09-01   2,000.0000
2020-09-02   2,014.9500
2020-09-03   1,965.7615
Name: total_value, dtype: float64

-----

### Trade Action

In [44]:
ticker1="ADP"
ticker2="APR"

today_trade = self_pair_trade(ticker1,ticker2,method = "realtimeday").iloc[-1]

log = get_pair_trade_log(ticker1,ticker2)

# day_diff = log["TimeStamp"].astimezone(mytz) - datetime.now(mytz)


if len(log) == 0:
    current_size1 = 0
    current_size2 = 0
else:
    log = log.iloc[-1]
    current_size1 = log["size1"]
    current_size2 = log["size2"]


new_size1 = today_trade["size1"]
new_size2 = today_trade["size2"]


trade_size1 = new_size1 - current_size1
trade_size2 = new_size2 - current_size2


try:
    td_trade = order_equity(order_type.LIMIT)

    ## need to trade the short sell first to test whether it's short
    if trade_size1 != 0:
        send_email(title="!Important! Pair Trade Order Placing for %s and %s"%(ticker1,ticker2))
        if trade_size1 < 0:
            td_trade.place(ticker1,trade_size1)
            
            td_trade.place(ticker2,trade_size2)
        if trade_size1 > 0:
            td_trade.place(ticker2,trade_size2)
            td_trade.place(ticker1,trade_size1)
        send_email("Trade process done.",title="!Important! Pair Trade Order Placed for %s and %s"%(ticker1,ticker2))
    else:
        print("No trade, skiped for %s and %s"%(ticker1,ticker2))
except Exception as e:
    send_email(str(e),title="!Important! Pair Trade Place Order Error")

All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')
All price data of Close is actually Adj Close
('Finished', 'ADP')
All price data of Close is actually Adj Close
('Finished', 'APR')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Connection Successful


In [3]:
mongod = mongo("all_symbol","pair_trade_sharp_2021_500")
candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}))

Connection Successful


In [7]:
candid[(candid.Ticker_1 == "ACA")&(candid.Ticker_2 == "DK")]

,_id,Ticker_1,Ticker_2,End_Value,Avg_Return,Sharp_Ratio,Min_Return,Max_Return,Refresh_Date
13355,612facb382f24db0e85f9777,ACA,DK,587.9126,0.0007,0.0707,-0.0254,0.0955,2021-08-31 00:00:00+00:00


In [50]:
ticker1="ACA"
ticker2="DK"
self_pair_trade(ticker1,ticker2,method = "realtimeday",cash = 500)

All price data of Close is actually Adj Close
('Finished', 'ACA')
All price data of Close is actually Adj Close
('Finished', 'DK')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


,relative,relative_mv,stock1,stock2,slope,hedge_ratio,z_score,up,mid,low,buy_line,sell_line,cash,buying_power,margin1,margin2,size1,size2,holddays,total_value
TimeStamp,,,,,,,,,,,,,,,,,,,,
2020-09-08,-0.0008,0.0008,46.4246,13.4300,-0.0088,0.2893,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-09,-0.0008,0.0008,46.4445,12.8400,-0.0088,0.2893,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-10,-0.0008,0.0008,44.4217,12.1500,-0.0088,0.2829,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-11,-0.0008,0.0008,44.3022,12.0600,-0.0088,0.2799,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
2020-09-14,-0.0008,0.0008,43.4950,12.2000,0.3151,0.2781,-0.5441,0.7507,-0.1280,-1.0066,-2.0000,2.0000,500.0000,500.0000,0.0000,0.0000,0,0,0,500.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-27,-0.0021,0.0027,52.2300,16.9900,2.1737,0.3158,-1.5822,-1.2590,-1.6421,-2.0251,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,0,599.4326
2021-08-30,-0.0018,0.0026,51.4900,16.9200,2.0676,0.3162,-1.4615,-1.2767,-1.6478,-2.0189,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,1,593.6526
2021-08-31,-0.0025,0.0027,50.8200,17.1100,2.1593,0.3167,-1.7166,-1.3257,-1.6828,-2.0400,-1.2500,1.2500,215.5726,111.1126,0.0000,104.4600,8,-2,2,587.9126


In [23]:
def cancel_order(orderid):
    return client.cancelOrder(277216702,orderid)

In [66]:
mongod = mongo("all_symbol","pair_trade_sharp_2021_500")
candid = pd.DataFrame(mongod.conn.table.find({"Sharp_Ratio":{"$exists":True}}).sort("Sharp_Ratio",-1).limit(10))

Connection Successful


In [2]:
get_open_opsition()

Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection

['QURE', 'DQ', 'QFIN', 'SVXY', 'WFG']

In [ ]:
log = get_pair_trade_log("KALV_CTLP")
log

In [11]:
log["size1"].sum()

47.0

In [12]:
log["size2"].sum()

-28.0

In [27]:
log = get_trade_log("VIXY")

Connection Successful


In [28]:
log["size"].sum()

0.0

In [29]:
log

,_id,Price,Strategy,Ticker,TimeStamp,size,raw_size
0,6029df6a797a11517a00688e,14.2700,vix_dayroll,VIXY,2019-11-18 13:04:18+00:00,21.0000,NaN
1,6029df6a797a11517a00688f,14.2700,vix_dayroll,VIXY,2019-11-18 13:15:14+00:00,21.0000,NaN
2,6029df6a797a11517a006890,14.3000,vix_dayroll,VIXY,2019-11-18 13:24:49+00:00,20.0000,NaN
3,6029df6a797a11517a006891,14.2700,vix_dayroll,VIXY,2019-11-19 07:03:59+00:00,21.0000,NaN
4,6029df6a797a11517a006892,287.0000,vix_dayroll,VIXY,2019-11-20 08:23:06+00:00,-20.0000,NaN
...,...,...,...,...,...,...,...
108,6206cede2881e001e3709317,18.9700,vix_dayroll,VIXY,2022-02-11 21:02:22.173000+00:00,-12.0000,-12.0000
109,620a6eceaf8d3b4c624a35d3,19.2500,vix_dayroll,VIXY,2022-02-14 15:01:34.265000+00:00,12.0000,12.0000
110,620a8ae79cdf5856121b3e89,19.0900,vix_dayroll,VIXY,2022-02-14 17:01:27.785000+00:00,-12.0000,-12.0000
111,620aae14d2ca95e0ab3ae59d,19.7100,vix_dayroll,VIXY,2022-02-14 19:31:32.023000+00:00,10.0000,10.0000


In [14]:
get_open_opsition()

Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection

['QURE', 'DQ', 'QFIN', 'SVXY', 'WFG']

In [43]:
today_trade = self_pair_trade(ticker1,ticker2,method = "realtimeday",cash = 500).iloc[-1]

ticker_combo = ticker1+"_"+ticker2
log = get_pair_trade_log(ticker_combo)

# day_diff = log["TimeStamp"].astimezone(mytz) - datetime.now(mytz)


if len(log) == 0:
    current_size1 = 0
    current_size2 = 0
else:
    
    current_size1 = log["size1"].sum()
    current_size2 = log["size2"].sum()


new_size1 = today_trade["size1"]
new_size2 = today_trade["size2"]


trade_size1 = new_size1 - current_size1
trade_size2 = new_size2 - current_size2

All price data of Close is actually Adj Close
('Finished', 'AA')
All price data of Close is actually Adj Close
('Finished', 'RUN')


/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:174: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/ken/miniconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


Connection Successful


In [44]:
trade_size1

-19.0

In [45]:
trade_size2

22.0

### DB manipulation

In [23]:
ticker = ticker1+"_"+ticker2
mongod = mongo("pair_trade_log",ticker)
mongod.conn.table.drop()

Connection Successful


In [4]:
mongod = mongo("pair_trade_log","pair_trade_log")

mongod.conn.db.list_collection_names()

Connection Successful


['AA_RUN']

In [24]:
mongod = mongo("all_symbol","td_not_shortable")
mongod.conn.table.insert_one({"TimeStamp":datetime.now(),"symbol":"APR"})

Connection Successful


In [25]:
pd.DataFrame(mongod.conn.table.find())

,_id,TimeStamp,symbol
0,612d38fc849e5662f18dc997,2021-08-30 13:01:00.435000+00:00,APR


In [19]:
client.accountsDF().loc[0,"currentBalances.availableFunds"]

441.99

In [ ]:
get